In [1]:
import yfinance as yf
import pandas as pd 


In [2]:
df = pd.read_excel('MCAP31032023_0.xlsx')
sym_comp = df['Symbol']
sym_comp.dropna(inplace=True)
sym_comp.isna().sum()
tickers = []
tickers.extend(sym_comp.values)

In [21]:
tickers[2]

'HDFCBANK'

In [22]:
data = yf.download('HDFCBANK',period='1y')

[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['HDFCBANK']: Exception('%ticker%: No data found, symbol may be delisted')


In [19]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [10]:
f = open('stocksdata.csv','w')
f.write(data)
# pd.tocsc=

SyntaxError: invalid syntax (1525091804.py, line 3)